Importing necessary libraries

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np
from pathlib import Path
from rasterio.windows import Window
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Reading geojson file
df = gpd.read_file('full_dataset/deforestation_labels.geojson')
df["img_date"] = gpd.pd.to_datetime(df["img_date"])

path = r"full_dataset\S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510\S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510.SAFE\GRANULE\L1C_T36UYA_A003350_20160212T084510\IMG_DATA\T36UYA_20160212T084052_B02.jp2"

# Setting the same coordinate system as in images in dataframe
with rasterio.open(path) as src:
    print("Image CRS:", src.crs)
    df = df.to_crs(src.crs)

# All Multipolygons in dataframe are divided into multiple polygons
df = df.explode(index_parts=False).reset_index(drop=True)

# Setting rectangle bounds for easier use
df[["minx", "miny", "maxx", "maxy"]] = df.geometry.bounds

Image CRS: EPSG:32636


In [3]:
df.head()

,img_date,tile,geometry,minx,miny,maxx,maxy
0,2016-04-09,36UXA,"POLYGON ((699307.129 5561713.193, 699414.418 5...",699246.875642,5.561537e+06,699432.480177,5.561713e+06
1,2016-04-09,36UXA,"POLYGON ((698548.1 5553743.058, 698658.019 555...",698548.100157,5.553503e+06,698686.443698,5.553799e+06
2,2016-04-09,36UXA,"POLYGON ((699613.313 5543770.618, 699605.429 5...",699605.429207,5.543771e+06,699921.463862,5.543889e+06
3,2016-04-09,36UXA,"POLYGON ((699203.094 5542952.982, 699333.117 5...",699202.900300,5.542791e+06,699333.116689,5.542958e+06
4,2016-04-09,36UXA,"POLYGON ((700515.258 5541902.106, 700605.663 5...",700515.258347,5.541572e+06,700606.448293,5.541921e+06


In [4]:
def find_band_files(base_dir, tile_id, date_str):
    """
    Returns dict of {band_name: path} for given tile_id and date (YYYYMMDD).
    """
    base_path = Path(base_dir)
    band_paths = {}
    for jp2 in base_path.rglob(f"T{tile_id}_{date_str}*_B*.jp2"):
        # jp2.name like T36UYA_20160212T084052_B04.jp2
        band_name = jp2.stem.split("_")[-1]  # 'B04'
        band_paths[band_name] = jp2
    return band_paths

In [ ]:
counterOne = 0
counterTwo = 0
cleaned_df = df.copy()

# There are many rows in dataframe that are not attached to images in folders we have at all. We must clean dataframe
for date in df["img_date"].unique():
    bands1 = find_band_files("full_dataset", "36UXA", date.strftime("%Y%m%d"))
    bands2 = find_band_files("full_dataset", "36UYA", date.strftime("%Y%m%d"))
    if bands1 and bands2:
        counterTwo += 1
    elif bands1 or bands2:
        counterOne += 1
    else:
        cleaned_df = cleaned_df[cleaned_df["img_date"] != date]

print(f"There are {counterOne} folders with date that are given in dataframe")
print(f"There are also {2 * counterTwo} folders more with date that are given in dataframe")
print(f"Now we have {cleaned_df.shape[0]} rows in dataset")

There are 42 folders with date that are given in dataframe
There are also 4 folders more with date that are given in dataframe
Now we have 1483 rows in dataset


In [ ]:
# This function is helpfull for acquaintance with dataset, but it is not used in this .ipynb file
def load_rgb_from_safe(base_dir, tile_id, date_str):
    """
    For a given tile and date finds B02, B03 and B04 files and making rgb array using them.
    """
    band_paths = find_band_files(base_dir, tile_id, date_str)
    with rasterio.open(band_paths["B04"]) as r:
        red = r.read(1).astype(np.float32)
        transform = r.transform 
    with rasterio.open(band_paths["B03"]) as g:
        green = g.read(1).astype(np.float32)
    with rasterio.open(band_paths["B02"]) as b:
        blue = b.read(1).astype(np.float32)
    rgb = np.stack([red, green, blue], axis=-1)
    return rgb, transform

In [ ]:
# Calculating area of polygons
cleaned_df["area_m2"] = cleaned_df.geometry.area

print(cleaned_df[["area_m2"]].describe())

            area_m2
count   1483.000000
mean    4159.749890
std     5746.969112
min        0.331636
25%      747.152385
50%     1893.633049
75%     5557.505381
max    51979.029961


In [ ]:
# Deletting all polygons that are too small
print(f"There were {cleaned_df.shape[0]} rows")

cleaned_df = cleaned_df[cleaned_df["area_m2"] > 500]

print(f"There are {cleaned_df.shape[0]} rows now")
print(cleaned_df[["area_m2"]].describe())

There were 1483 rows
There are 1244 rows now
            area_m2
count   1244.000000
mean    4898.574829
std     5998.808731
min      501.777945
25%     1180.015200
50%     2509.856928
75%     6679.559261
max    51979.029961


In [ ]:
def load_rgb_crop(base_dir, tile_id, date_str, minx, maxx, miny, maxy, size=32):
    """
    Loads a 32x32 RGB crop (B04, B03, B02) from Sentinel-2 data.
    The crop is centered in the given bounding box (map coordinates).
    """
    # Find band file paths
    bands = find_band_files(base_dir, tile_id, date_str)
    
    # Get center coordinates of bbox
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2
    
    rgb_bands = []
    
    for band_name in ["B04", "B03", "B02"]:  # R, G, B
        path = bands[band_name]
        with rasterio.open(path) as src:
            width, height = src.width, src.height
            col, row = src.index(center_x, center_y)
            half = size // 2
            
            # Compute window boundaries (clip to image size)
            col_start = max(col - half, 0)
            row_start = max(row - half, 0)
            col_end = min(col + half, width)
            row_end = min(row + half, height)
            
            if col_end < col_start:
                col_start, col_end = col_end, col_start
            if row_end < row_start:
                row_start, row_end = row_end, row_start

            w = col_end - col_start
            h = row_end - row_start

            if w <= 0 or h <= 0:
                continue

            window = Window(col_start, row_start, w, h)
            
            # Read one band crop
            patch = src.read(1, window=window)

            valid_pixels = np.count_nonzero(patch)
            if valid_pixels < 1000:
                return None 

            # Pad if at edge
            if patch.shape[0] < size or patch.shape[1] < size:
                pad_h = size - patch.shape[0]
                pad_w = size - patch.shape[1]
                patch = np.pad(patch, ((0, pad_h), (0, pad_w)), mode="reflect")
            
            rgb_bands.append(patch)
    
    # Stack bands → (3, size, size)
    rgb = np.stack(rgb_bands, axis=0).astype(np.float32)
    
    return rgb

In [ ]:
def robust_percentile_normalize(rgb, lower=1, upper=99):
    """
    Percentile normalization across each channel of an RGB patch.
    Ignores NaNs and zeros. Returns array in [0, 1].
    """
    norm = np.zeros_like(rgb, dtype=np.float32)

    for i in range(rgb.shape[0]):
        band = rgb[i].astype(np.float32)

        # Mask invalid values (0 and NaN)
        valid = band[np.isfinite(band) & (band > 0)]
        if valid.size == 0:
            norm[i] = 0
            continue

        # Calculating percentiles
        p1, p99 = np.percentile(valid, [lower, upper])
        if not np.isfinite(p1) or not np.isfinite(p99) or (p99 - p1) < 1e-6:
            # Fallback to min-max
            vmin, vmax = valid.min(), valid.max()
            if vmax - vmin < 1e-6:
                norm[i] = 0
                continue
            p1, p99 = vmin, vmax

        norm[i] = np.clip((band - p1) / (p99 - p1), 0, 1)

    return norm


In [ ]:
# We buffer our polygons by 10m. We grab some surroundings by this
df_buffered = cleaned_df.copy()
df_buffered["geometry"] = df_buffered.buffer(10)

# We find all polygons that intersects. So those are potentially the same polygon over some time
joined = gpd.sjoin(df_buffered, df_buffered, predicate="intersects", how="inner")

# Keep only same-tile matches but different dates
joined = joined[joined["tile_left"] == joined["tile_right"]]
joined = joined[joined["img_date_left"] != joined["img_date_right"]]

In [12]:
print(f"There were found {joined.shape[0]} positive pairs (same location, but different time)")

There were found 2582 positive pairs (same location, but different time)


In [ ]:
positive_pairs = []
labels = []

# Finding, reading and storing all positive pairs (pairs of images with the same polygon but in different times)
for _, row in tqdm(joined.iterrows(), total=len(joined)):
    try:
        minx1, miny1, maxx1, maxy1 = row[["minx_left", "miny_left", "maxx_left", "maxy_left"]]
        minx2, miny2, maxx2, maxy2 = row[["minx_right", "miny_right", "maxx_right", "maxy_right"]]
        
        img1 = load_rgb_crop("full_dataset", row["tile_left"], row["img_date_left"].strftime("%Y%m%d"), minx1, maxx1, miny1, maxy1)
        img2 = load_rgb_crop("full_dataset", row["tile_right"], row["img_date_right"].strftime("%Y%m%d"), minx2, maxx2, miny2, maxy2)

        if img1 is None or img2 is None:
            continue

        img1 = robust_percentile_normalize(img1)
        img2 = robust_percentile_normalize(img2)

        positive_pairs.append((img1, img2))
        labels.append(1)
    except Exception as e:
        print("Skipping pair:", e)

100%|██████████| 2582/2582 [09:04<00:00,  4.74it/s]


In [14]:
print(f"There were {len(positive_pairs)} pairs generated for dataset.")

There were 2300 pairs generated for dataset.


In [ ]:
# Making sure that all pairs are from different locations
df_UXA = cleaned_df[cleaned_df["tile"] == "36UXA"]
df_UYA = cleaned_df[cleaned_df["tile"] == "36UYA"]

# Getting some subsets
UXA_subset = df_UXA.sample(120, random_state = 50)
UYA_subset = df_UYA.sample(120, random_state = 50)

# Creating negative pairs (images with different polygons)
pairs1 = UXA_subset.iloc[0:40].merge(UYA_subset.iloc[0:40], how="cross", suffixes=("_left", "_right"))
pairs2 = UXA_subset.iloc[40:80].merge(UYA_subset.iloc[40:80], how="cross", suffixes=("_left", "_right"))
pairs3 = UXA_subset.iloc[80:120].merge(UYA_subset.iloc[80:120], how="cross", suffixes=("_left", "_right"))
pairs = pd.concat([pairs1, pairs2, pairs3])

In [ ]:
negative_pairs = []

# Finding, reading and storing all negative pairs (images with different polygons)
for _, row in tqdm(pairs.iterrows(), total=len(pairs)):
    try:
        minx1, miny1, maxx1, maxy1 = row[["minx_left", "miny_left", "maxx_left", "maxy_left"]]
        minx2, miny2, maxx2, maxy2 = row[["minx_right", "miny_right", "maxx_right", "maxy_right"]]
        
        img1 = load_rgb_crop("full_dataset", row["tile_left"], row["img_date_left"].strftime("%Y%m%d"), minx1, maxx1, miny1, maxy1)
        img2 = load_rgb_crop("full_dataset", row["tile_right"], row["img_date_right"].strftime("%Y%m%d"), minx2, maxx2, miny2, maxy2)

        if img1 is None or img2 is None:
            continue

        img1 = robust_percentile_normalize(img1)
        img2 = robust_percentile_normalize(img2)

        negative_pairs.append((img1, img2))
        labels.append(0)
    except Exception as e:
        print("Skipping pair:", e)

100%|██████████| 4800/4800 [16:40<00:00,  4.80it/s]


In [ ]:
# Combining positive pairs with negative pairs
all_pairs = positive_pairs + negative_pairs

In [42]:
print(f"There are {len(all_pairs)} pairs for dataset")

There are 5913 pairs for dataset


In [ ]:
# Splitting data into train, test and validation parts (80/10/10)
pairs_train, pairs_test_val, labels_train, labels_test_val = train_test_split(
    all_pairs, labels, test_size=0.2, random_state=50, stratify=labels
)

pairs_test, pairs_val, labels_test, labels_val = train_test_split(
    pairs_test_val, labels_test_val, test_size=0.5, random_state=50, stratify=labels_test_val
)

In [ ]:
# Saving so we don't need to generate dataset again
np.savez("pairs_train_data.npz", pairs=pairs_train, labels=labels_train)
np.savez("pairs_test_data.npz", pairs=pairs_test, labels=labels_test)
np.savez("pairs_validation_data.npz", pairs=pairs_val, labels=labels_val)